In [293]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.semi_supervised import LabelPropagation
plt.rcParams['figure.figsize'] = [12, 10]

In [34]:
class LinearRegressionClassifier(BaseEstimator, ClassifierMixin):  
    """An example of classifier"""

    def __init__(self,reg):
        self.reg_ = reg


    def fit(self, X, y=None):

        self.X_ = X
        self.y_ = y
        self.threshold_ = 0  
        self.reg_.fit(X,y)
        return self

    def predict(self, X):
        try:
            getattr(self, "threshold_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        
        predictions = self.reg_.predict(X)
        predictions[predictions > self.threshold_] = 1
        predictions[predictions < self.threshold_] = -1
        
        return predictions
    


In [177]:
def self_learning(L, U, y,k):
    reg = LinearRegression()
    classifier = LinearRegressionClassifier(reg)
    max_iter = 10000
    t = 0
    while U.shape[0] > 0 and t < max_iter:
        classifier.fit(L,y)
        y_pred = np.abs(classifier.reg_.predict(U))
        y_class = classifier.predict(U)
        indexes = (-y_pred).argsort()[:k]
        L = np.vstack([L, U[indexes,:]])
        U = np.delete(U, indexes, 0)       
        y = np.append(y, y_class[indexes])
        t = t + 1
    classifier.fit(L,y)
    return classifier

In [188]:
def estimate_error(X_test,y_test,classifier):
    y_pred = classifier.predict(X_test)
    equal = np.sum(y_pred == y_test)
    error = 1-(equal/y_test.shape[0])
    return error

In [281]:
def estimate_loss(X_test,y_test,classifier):
    y_pred_cont = classifier.reg_.predict(X_test)
    losses = (y_test - y_pred_cont)**2
    loss = np.sum(losses)/X_test.shape[0]
    return loss

In [311]:
def learning_curve(mean,std,n_unlabeled,ylab,title):
    plt.title(title)
    plt.plot(n_unlabeled,mean)
    plt.ylabel(ylab)
    plt.xlabel('Number of Unlabeled Samples')
    plt.errorbar(n_unlabeled, mean, std, linestyle='None', marker='^')
    plt.axhline(mean[0],linestyle='--',color='red',label="Error Rate Supervised")
    plt.show()

In [275]:
def label_propagation(X_u, y_l, X_l,k):
    X_train_prop = np.concatenate((X_l, X_u), axis=0)
    labels = np.concatenate((y_l,np.repeat(-1, X_u.shape[0])))
    label_prop_model = LabelPropagation(kernel='knn',n_neighbors=k)
    label_prop_model.fit(X_train_prop, labels)
    y_train = label_prop_model.transduction_
    X_train = np.concatenate((X_l, X_u), axis=0)
    return X_train, y_train